# Moments

**Definition: Moments** *If $X$ is a random variable, the rth moment of $X$, usually denoted by $\mu_r'$ is defined as:*

$$\mu_r' = \mathscr E\left[X^r\right]$$

Trying to give an interpretation to each movement, for that I use UC distribution:

In [1]:
# Creating an interactive horizontal subplot showing:
# 1) a function X(x) = a*x + b (linear by default)
# 2) a probability density function (selectable)
# 3) the product X(x) * pdf(x)
# Also computes raw moment mu_r' = E[X^r] for an adjustable r (here E is w.r.t. the pdf on the chosen support).
#
# This runs in the notebook environment and uses matplotlib + ipywidgets.
# If ipywidgets doesn't render automatically in your environment, try running this in a Jupyter notebook/lab
# with the ipywidgets extension enabled.
#
# NOTE: charts use matplotlib (no seaborn) and each chart is on its own axis (no shared colors specified).

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, uniform
from ipywidgets import interact, interactive_output, FloatSlider, Dropdown, IntSlider, VBox, HBox, fixed, Checkbox
from IPython.display import display, HTML

def compute_moment(x, pdf, r):
    # raw moment E[X^r] using numeric integration
    integrand = (x**r) * pdf
    return round(np.trapezoid(integrand, x),2)

def update(dist, xmin, xmax,
           r, # moment order
           # distribution params
           mu, sigma, lam, normalize_display):
    x = np.linspace(-10, 10, 2000)
    Xx = x**r
    
    # choose pdf
    if dist == "Uniform":
        pdf = uniform.pdf(x, loc=round(xmin,0), scale=(round(xmax,0) - round(xmin,0)))
        dist_info = f"Uniform on [{round(xmin,0)}, {round(xmax,0)}]"
    elif dist == "Normal":
        pdf = norm.pdf(x,round(mu,2),round(sigma,2))
        dist_info = f"Truncated Normal μ={round(mu,2)}, σ={round(sigma,2)}"
    elif dist == "Exponential":
        pdf = expon(x, round(lam,2))
        dist_info = f"Shifted Exponential λ={round(lam,2)}"
    else:
        pdf = uniform.pdf(x, loc=round(xmin,0), scale=(round(xmax,0) - round(xmin,0)))
        dist_info = "Uniform (fallback)"
    
    # compute product: here we interpret "multiplication of this two" as X(x)*pdf(x)
    product = Xx * pdf
    
    # compute raw moments numerically for X (the random variable is the *domain variable* x)
    mu_r = compute_moment(x, pdf, r)
    mu_1 = compute_moment(x, pdf, 1)
    mu_2 = compute_moment(x, pdf, 2)
    
    # plotting - horizontal (1 row, 3 columns)
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1) function X(x)
    axes[0].plot(x, Xx)
    axes[0].set_title(f"Function X(x) = x^{r}")
    axes[0].set_xlabel("x")
    axes[0].set_ylabel("X(x)")
    axes[0].set_xlim(-10,10)
    axes[0].set_ylim(-10,10)
    axes[0].grid(ls ='dashed', lw = 0.5)
    
    # 2) pdf
    axes[1].plot(x, pdf)
    axes[1].set_title("Probability density function (pdf)")
    axes[1].set_xlabel("x")
    axes[1].set_ylabel("pdf(x)")
    axes[1].text(0.02, 0.95, dist_info, transform=axes[1].transAxes, va='top', ha='left', fontsize=9)
    axes[1].set_xlim(-10,10)
    axes[1].grid(ls ='dashed', lw = 0.5)
    
    # 3) product
    axes[2].plot(x, product)
    axes[2].set_title("Product X(x) * pdf(x)")
    axes[2].set_xlabel("x")
    axes[2].set_ylabel("X(x) * pdf(x)")
    axes[2].set_xlim(-10,10)
    axes[2].grid(ls ='dashed', lw = 0.5)
    
    plt.tight_layout()
    plt.show()
    
    # display computed moments
    display(HTML(f"<b>Computed raw moments (numerical)</b>: E[x] = {mu_1:.2g}, E[x^2] = {mu_2:.2g}, E[x^{r}] = {mu_r:.2g}"))
    display(HTML("<i>Note:</i> moments are with respect to the pdf shown (domain variable is x)."))
    

# Create widgets
dist_widget = Dropdown(options=["Uniform", "Normal", "Exponential"],
                       value="Uniform", description="Distribution:")

xmin_slider = FloatSlider(value=0, min=-10, max=10, step=1, description="xmin:")
xmax_slider = FloatSlider(value=1, min=-10, max=10, step=1, description="xmax:")

r_slider = IntSlider(value=1, min=0, max=6, step=1, description="r (moment):")

# distribution-specific widgets
mu_slider = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description="μ:")
sigma_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="σ:")
lam_slider = FloatSlider(value=1.0, min=0.01, max=5.0, step=0.01, description="λ:")

# link xmax min constraint: ensure xmax > xmin visually (user can still set equal but grid will be fine)
def on_xmin_change(change):
    # ensure xmax slider min is slightly above xmin
    new_min = change['new'] + 0.01
    if xmax_slider.min <= new_min:
        xmax_slider.min = new_min

xmin_slider.observe(on_xmin_change, names='value')

ui_left = VBox([dist_widget, VBox([xmin_slider, xmax_slider])])
ui_right = VBox([r_slider])
ui_params = VBox([mu_slider, sigma_slider, lam_slider])

ui = HBox([ui_left, ui_right, ui_params])

out = interactive_output(update, {
    'dist': dist_widget,
    'xmin': xmin_slider,
    'xmax': xmax_slider,
    'r': r_slider,
    'mu': mu_slider,
    'sigma': sigma_slider,
    'lam': lam_slider,
    'normalize_display': fixed(True)
})

display(ui, out)


Output()

**Definition: Central moments** *If $X$ is a random variable, the rth central moment of $X$ about $a$ is defined as*

$$\mathscr E\left[\left(X - a\right)^r\right]$$

*If $a = \mu_X$, we have the rth central moment of $X$ about $\mu_X$, denored by $\mu_r$, which is:*

$$\mu_r = \mathscr E\left[\left(X - \mu_X\right)^r\right]$$



In [4]:
# Creating an interactive horizontal subplot showing:
# 1) a function X(x) = a*x + b (linear by default)
# 2) a probability density function (selectable)
# 3) the product X(x) * pdf(x)
# Also computes raw moment mu_r' = E[X^r] for an adjustable r (here E is w.r.t. the pdf on the chosen support).
#
# This runs in the notebook environment and uses matplotlib + ipywidgets.
# If ipywidgets doesn't render automatically in your environment, try running this in a Jupyter notebook/lab
# with the ipywidgets extension enabled.
#
# NOTE: charts use matplotlib (no seaborn) and each chart is on its own axis (no shared colors specified).

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, uniform
from ipywidgets import interact, interactive_output, FloatSlider, Dropdown, IntSlider, VBox, HBox, fixed, Checkbox
from IPython.display import display, HTML

def compute_moment(Xx, pdf, r):
    # raw moment E[X^r] using numeric integration
    integrand = Xx* pdf
    return round(np.trapezoid(integrand, Xx),2)

def update(dist, xmin, xmax,
           r, # moment order
           # distribution params
           mu, sigma, lam, normalize_display):
    x = np.linspace(-10, 10, 2000)
    
    # choose pdf
    if dist == "Uniform":
        pdf = uniform.pdf(x, loc=round(xmin,0), scale=(round(xmax,0) - round(xmin,0)))
        mean = uniform.mean(loc=round(xmin,0), scale=(round(xmax,0) - round(xmin,0)))
        dist_info = f"Uniform on [{round(xmin,0)}, {round(xmax,0)}]"
    elif dist == "Normal":
        pdf = norm.pdf(x,round(mu,2),round(sigma,2))
        mean = norm.mean(round(mu,2),round(sigma,2))
        dist_info = f"Truncated Normal μ={round(mu,2)}, σ={round(sigma,2)}"
    elif dist == "Exponential":
        pdf = expon.pdf(x, round(lam,2))
        mean = expon.mean(round(lam,2))
        dist_info = f"Shifted Exponential λ={round(lam,2)}"
    else:
        pdf = uniform.pdf(x, loc=round(xmin,0), scale=(round(xmax,0) - round(xmin,0)))
        mean = uniform.mean(loc=round(xmin,0), scale=(round(xmax,0) - round(xmin,0)))
        dist_info = "Uniform (fallback)"

    Xx = (x-mean)**r
    
    # compute product: here we interpret "multiplication of this two" as X(x)*pdf(x)
    product = Xx * pdf
    
    # compute raw moments numerically for X (the random variable is the *domain variable* x)
    mu_r = compute_moment(Xx, pdf, r)
    mu_1 = compute_moment((x-mean), pdf, 1)
    mu_2 = compute_moment((x-mean)**2, pdf, 2)
    
    # plotting - horizontal (1 row, 3 columns)
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # 1) function X(x)
    axes[0].plot(x, x**r, alpha = 0.1, c = 'red')
    axes[0].plot(x, Xx)
    axes[0].set_title(f"Function X(x) = (x - $\mu$)^{r}")
    axes[0].set_xlabel("x")
    axes[0].set_ylabel("X(x)")
    axes[0].set_xlim(-10,10)
    axes[0].set_ylim(-10,10)
    axes[0].grid(ls ='dashed', lw = 0.5)
    
    # 2) pdf
    axes[1].plot(x, pdf)
    axes[1].set_title("Probability density function (pdf)")
    axes[1].set_xlabel("x")
    axes[1].set_ylabel("pdf(x)")
    axes[1].text(0.02, 0.95, dist_info, transform=axes[1].transAxes, va='top', ha='left', fontsize=9)
    axes[1].set_xlim(-10,10)
    axes[1].grid(ls ='dashed', lw = 0.5)
    
    # 3) product
    axes[2].plot(x, x**r*pdf, alpha = 0.1, c = 'red')
    axes[2].plot(x, product)
    axes[2].set_title("Product X(x) * pdf(x)")
    axes[2].set_xlabel("x")
    axes[2].set_ylabel("X(x) * pdf(x)")
    axes[2].set_xlim(-10,10)
    axes[2].grid(ls ='dashed', lw = 0.5)
    
    plt.tight_layout()
    plt.show()
    
    # display computed moments
    display(HTML(f"<b>Computed raw moments (numerical)</b>: E[x] = {mu_1:.2g}, E[x^2] = {mu_2:.2g}, E[x^{r}] = {mu_r:.2g}"))
    display(HTML("<i>Note:</i> moments are with respect to the pdf shown (domain variable is x)."))
    

# Create widgets
dist_widget = Dropdown(options=["Uniform", "Normal", "Exponential"],
                       value="Uniform", description="Distribution:")

xmin_slider = FloatSlider(value=0, min=-10, max=10, step=1, description="xmin:")
xmax_slider = FloatSlider(value=1, min=-10, max=10, step=1, description="xmax:")

r_slider = IntSlider(value=1, min=0, max=6, step=1, description="r (moment):")

# distribution-specific widgets
mu_slider = FloatSlider(value=0.0, min=-5.0, max=5.0, step=0.1, description="μ:")
sigma_slider = FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="σ:")
lam_slider = FloatSlider(value=1.0, min=0.01, max=5.0, step=0.01, description="λ:")

# link xmax min constraint: ensure xmax > xmin visually (user can still set equal but grid will be fine)
def on_xmin_change(change):
    # ensure xmax slider min is slightly above xmin
    new_min = change['new'] + 0.01
    if xmax_slider.min <= new_min:
        xmax_slider.min = new_min

xmin_slider.observe(on_xmin_change, names='value')

ui_left = VBox([dist_widget, VBox([xmin_slider, xmax_slider])])
ui_right = VBox([r_slider])
ui_params = VBox([mu_slider, sigma_slider, lam_slider])

ui = HBox([ui_left, ui_right, ui_params])

out = interactive_output(update, {
    'dist': dist_widget,
    'xmin': xmin_slider,
    'xmax': xmax_slider,
    'r': r_slider,
    'mu': mu_slider,
    'sigma': sigma_slider,
    'lam': lam_slider,
    'normalize_display': fixed(True)
})

display(ui, out)


Output()

*Interpretation:*

For the **zero-th moment**, nothing about the actual distribution matters except that it integrates to one. It does not put any weight on the variable values themselves; it is purely a statement that probability mass is normalized.

When we move to the **first moment**, now the variable values begin to matter: each outcome is weighted by its associated probability. This is why the first moment corresponds to the mean — it is the value around which the probability “balances.”

At the **second moment**, the contributions grow with the square of the distance from zero. Outcomes further away from the origin contribute disproportionately more, so it's a measure that tries to capture how **spread** the values are from zero. This is why, once centered at the mean, the second moment becomes the variance: it measures how spread out the distribution is, how far typical values lie from the central location.

With the **third moment**, the sign of the deviation comes into play. Positive values far from the mean push the moment upward, while negative values far from the mean pull it downward. A negative third moment indicates that the left tail of the distribution is heavier, while a positive one signals that the right tail is heavier. In other words, the third moment captures the **skewness** of the distribution — whether probability mass leans more to the left or to the right.

At the **fourth moment**, distance takes on an even greater role. Values that lie far away from the mean contribute not just more, but overwhelmingly more, because the deviation is raised to the fourth power. This moment is sensitive to the tails of the distribution: whether probability is concentrated tightly around the mean or spread into rare but extreme values. Once standardized, it yields the **kurtosis**, which distinguishes between thin-tailed and heavy-tailed behaviors.